In [62]:
import torch.nn as nn
import torch
import torchmetrics.functional as F
from torchvision import models, datasets, transforms
from torch.utils.data import Dataset, DataLoader, random_split, TensorDataset

import lightning as L

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

train_data = pd.read_csv('data/train.csv')
test_data = pd.read_csv('data/test.csv')

X = train_data.iloc[:, 1:]
y = train_data.label.values

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

In [63]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

X_train = torch.from_numpy(X_train)
y_train = torch.from_numpy(y_train)

X_val = torch.from_numpy(X_val)
y_val = torch.from_numpy(y_val)

train_ds = TensorDataset(X_train, y_train)
valid_ds = TensorDataset(X_val, y_val)

train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)
val_loader = DataLoader(valid_ds, batch_size=64, shuffle=False)

In [125]:
class MNISTNet(L.LightningModule):
    def __init__(self, lr=1e-3):
        super().__init__()

        self.lr = lr
        self.model = nn.Sequential(nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1),
                                   nn.ReLU(),
                                   nn.MaxPool2d(kernel_size=2, stride=2),
                                   nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
                                   nn.ReLU(),
                                   nn.MaxPool2d(kernel_size=2, stride=2),
                                   nn.Flatten(),
                                   nn.Linear(32 * 7 * 7, 120),
                                   nn.ReLU(),
                                   nn.Linear(120, 60),
                                   nn.ReLU(),
                                   nn.Linear(60, 10))
        self.loss = nn.CrossEntropyLoss()

    def forward(self, x):
        return self.model(x)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        return optimizer

    def _shared_step(self, batch, prefix):
        x, y = batch
        x = x.view(x.size(0), 1, 28, 28)
        x = x.to(self.model[0].weight.dtype)
        y_pred = self.forward(x)

        loss = self.loss(y_pred, y)
        acc = F.accuracy(y_pred, y, task='multiclass', num_classes=10)

        self.log(f'{prefix}_loss', loss)
        self.log(f'{prefix}_acc', acc)

        return loss, acc

    def training_step(self, train_batch, batch_idx):
        loss, acc = self._shared_step(train_batch, 'train')
        return loss

    def validation_step(self, val_batch, batch_idx):
        loss, acc = self._shared_step(val_batch, 'val')
        return loss

In [45]:
!wandb login

wandb: Currently logged in as: devour (devour-team). Use `wandb login --relogin` to force relogin


In [126]:
from pytorch_lightning.loggers import WandbLogger
import wandb

wandb_logger = WandbLogger(project='mnist')
wandb_logger.experiment.config['train_batch_size'] = 128
wandb_logger.experiment.config['val_batch_size'] = 64

model = MNISTNet()
trainer = L.Trainer(max_epochs=25, logger=wandb_logger)
trainer.fit(model, train_loader, val_loader)

wandb.finish()

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type             | Params
-------------------------------------------
0 | model | Sequential       | 200 K 
1 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
200 K     Trainable params
0         Non-trainable params
200 K     Total params
0.804     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

C:\Users\Andrei\anaconda3\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:442: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
C:\Users\Andrei\anaconda3\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:442: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=25` reached.


epoch,▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇███
train_acc,▁▅▅▇▇▇▇▇██▇▇████████████████████████████
train_loss,█▄▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
trainer/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val_acc,▁▆▇▆▇▇▇█▇▇██▇█▇█████▇█▇██
val_loss,█▃▂▂▃▁▁▁▂▂▁▁▂▁▂▁▁▁▁▁▂▂▃▁▂
epoch,24
train_acc,1.0
train_loss,0.00137
trainer/global_step,5749
val_acc,0.98706


In [127]:
X_test = test_data.values
X_test = torch.from_numpy(scaler.transform(X_test))

y_test = np.zeros(X_test.shape)
y_test = torch.from_numpy(y_test)

test_ds = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_ds, batch_size=128, shuffle=False)

C:\Users\Andrei\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [132]:
submission_rows = [['ImageId', 'Label']]

with torch.no_grad():
    model.eval()
    image_id = 1

    for X, _ in test_loader:
        X = (X.view(-1, 1, 28, 28)).type(torch.FloatTensor).to('cpu')
        preds = model(X).argmax(1)

        for pred in preds:
            submission_rows.append([image_id, pred.item()])
            image_id += 1

submission = pd.DataFrame(submission_rows)
submission.columns = submission.iloc[0]
submission = submission.drop(0, axis=0)

submission.to_csv('submission.csv', index=False)